In [1]:
import random as rd
import numpy as np
import h5py
import klepto
import shelve
import pickle
import scipy.io as sio
from scipy.sparse import *

## Input Prediction Model

In [9]:
import shelve
import pickle
prediction_model = {}
tree_num = 3
dataset = '1m'

for i in range(tree_num):
    with shelve.open("./s%s/P_test"%(str(i+1)), protocol=pickle.HIGHEST_PROTOCOL) as d:
        P_test_tree = d['content'].tocsc().toarray()
    with shelve.open("./s%s/rated_matrix"%(str(i+1)), protocol=pickle.HIGHEST_PROTOCOL) as d:
        rated_matrix_tree = d['content']
    prediction_model[i] = {'P_test': P_test_tree, 'rated_matrix': rated_matrix_tree}

## Input Test item info

In [4]:
feature=h5py.File('test_list.mat')  
test_list = feature['test_list'][:]
test_list = list(map(int, test_list.T[0]))
test_list = [i-1 for i in test_list] 
feature=h5py.File('train_list.mat')
train_list = feature['train_list'][:]
train_list = list(map(int, train_list.T[0]))
train_list = [i-1 for i in train_list]

In [6]:
rating_matrix = load_npz('sparse_matrix_ml-' + dataset + '_selected.npz').tocsc()
rating_matrix_train = rating_matrix[:, train_list]
rating_matrix_test  = rating_matrix[:, test_list]
print("file load DONE")
rating_matrix_test = rating_matrix_test.toarray()

file load DONE


## Aggregate Three Tree models

In [13]:
tree_res = {}
for i in range(tree_num):
    tree_res[i] = []
for itemid in range(rating_matrix_test.shape[1]):
    model_ind = rd.randint(0, tree_num-1)
    tree_res[model_ind].append(itemid)
    
P_test = {}
rated_user = {}
rating_matrix_test_unqueried = {}
for i in range(tree_num):
    P_test[i] = prediction_model[i]['P_test'][:, tree_res[i]]
    rated_user[i] = prediction_model[i]['rated_matrix'][:, tree_res[i]]
    rating_matrix_test_unqueried[i] = rating_matrix_test[:, tree_res[i]] * rated_user[i]

RMSE_nominator = 0
RMSE_denominator = 0
Precision_nominator = 0
Precision_denominator = 0
for i in range(tree_num):
    dif = P_test[i]*(rating_matrix_test_unqueried[i]!=0) - rating_matrix_test_unqueried[i]
    RMSE_nominator += (dif**2).sum()
    RMSE_denominator += (rating_matrix_test_unqueried[i]!=0).sum()
    Precision_nominator += np.multiply(P_test[i]>3, rating_matrix_test_unqueried[i]>3).sum()
    Precision_denominator += (rating_matrix_test_unqueried[i]>3).sum()
    
RMSE = ( RMSE_nominator/RMSE_denominator )**0.5
Precision =  Precision_nominator/Precision_denominator

In [14]:
Precision

0.92263236067525467

In [15]:
RMSE

0.92350529194348741